
# 🧠 FASE 2 – Análise de Sentimentos em Comentários do YouTube  
**Projeto:** Análise de Sentimentos em Comentários de Vídeos  
**Etapa do CRISP-DM:** Modeling  

Este notebook carrega os comentários coletados (Fase 1), realiza **limpeza e normalização de texto**, faz **detecção de idioma** e aplica um **modelo pré-treinado em português** (HuggingFace) para classificar os sentimentos em **positivo, negativo e neutro**.  
Resultados são exportados para CSV e gráficos básicos são gerados para suporte à Fase 3 (Visualização/EDA).
    

In [ ]:

# ============================================================
# 1️⃣ Importação das bibliotecas
# ============================================================
import os, re, time
import pandas as pd
import matplotlib.pyplot as plt
from langdetect import detect, DetectorFactory
from transformers import pipeline
import emoji

# Reprodutibilidade na detecção de idioma
DetectorFactory.seed = 0

print("✅ Bibliotecas carregadas.")
    

In [ ]:

# ============================================================
# 2️⃣ Carregar os dados da Fase 1
# ============================================================
INPUT_PATH = os.getenv("INPUT_PATH", "comentarios_top5.csv")
assert os.path.exists(INPUT_PATH), f"Arquivo não encontrado: {INPUT_PATH}"

df = pd.read_csv(INPUT_PATH)
print(f"✅ Dataset carregado: {INPUT_PATH}")
print(f"Total de registros: {len(df)}")
display(df.head(3))
    

In [ ]:

# ============================================================
# 3️⃣ Limpeza e normalização de texto
#  - demojize: emojis viram tokens textuais (:smile:)
#  - remove URLs, menções e caracteres não textuais
#  - normaliza espaços e caixa
# ============================================================
def limpar_texto(texto: str) -> str:
    texto = str(texto)
    texto = emoji.demojize(texto)
    texto = re.sub(r"http\S+|www\.\S+", " ", texto)        # URLs
    texto = re.sub(r"@[A-Za-z0-9_]+", " ", texto)             # menções
    texto = re.sub(r"#[A-Za-z0-9_]+", " ", texto)             # hashtags (opcional)
    texto = re.sub(r"[^A-Za-zÀ-ÿ\s:]", " ", texto)           # mantém letras/acentos/espaco/:
    texto = re.sub(r"\s+", " ", texto).strip().lower()
    return texto

df["comentario_limpo"] = df["comentario"].astype(str).apply(limpar_texto)
df = df[df["comentario_limpo"].str.len() > 3].copy()  # remove muito curtos
print("🧹 Limpeza concluída.")
display(df.head(3))
    

In [ ]:

# ============================================================
# 4️⃣ Detecção de idioma e filtragem (PT-first)
#    - langdetect falha em textos curtos; tratamos exceções
#    - filtro padrão: manter pt; demais idiomas podem ser analisados depois
# ============================================================
def detectar_idioma_seguro(texto: str) -> str:
    try:
        return detect(texto)
    except Exception:
        return "indefinido"

df["idioma"] = df["comentario_limpo"].apply(detectar_idioma_seguro)
df_pt = df[df["idioma"] == "pt"].copy()

print(f"Total após limpeza: {len(df)}")
print(f"Total em português (pt): {len(df_pt)}")
display(df_pt.head(3))
    

In [ ]:

# ============================================================
# 5️⃣ Carregar pipeline de análise de sentimentos (PT-BR)
#    - Dica de performance: carregar uma vez; usar truncation e batch
# ============================================================
MODEL_NAME = os.getenv("HF_MODEL_PT", "pierreguillou/bert-base-cased-sentiment-br")

sentiment_pt = pipeline(
    task="sentiment-analysis",
    model=MODEL_NAME,
    truncation=True,
    batch_size=32
)

print(f"🧠 Modelo carregado: {MODEL_NAME}")
    

In [ ]:

# ============================================================
# 6️⃣ Inferência em lote (batch) e padronização de rótulos
# ============================================================
comentarios = df_pt["comentario_limpo"].tolist()
resultados = sentiment_pt(comentarios)

# Converte para DataFrame e agrega
res_df = pd.DataFrame(resultados)
# Mapa de rótulos para português
label_map = {
    "POS": "positivo", "positive": "positivo",
    "NEG": "negativo", "negative": "negativo",
    "NEU": "neutro",   "neutral":  "neutro"
}
df_pt["sentimento"] = res_df["label"].map(label_map).fillna(res_df["label"])
df_pt["score"] = res_df["score"]

print("✅ Inferência concluída.")
display(df_pt.head(5))
    

In [ ]:

# ============================================================
# 7️⃣ Gráficos básicos (suporte à Fase 3)
#    - Matplotlib puro (sem seaborn) para compatibilidade
# ============================================================
# Distribuição geral dos sentimentos
dist = df_pt["sentimento"].value_counts().sort_index()
plt.figure(figsize=(6,4))
dist.plot(kind="bar")
plt.title("Distribuição de Sentimentos (PT)")
plt.xlabel("Sentimento")
plt.ylabel("Quantidade")
plt.tight_layout()
plt.show()

# Comparativo por vídeo (empilhado simples)
sent_por_video = df_pt.groupby(["video_titulo", "sentimento"]).size().unstack(fill_value=0)
plt.figure(figsize=(10,5))
sent_por_video.plot(kind="bar", stacked=True)
plt.title("Sentimentos por Vídeo (PT)")
plt.xlabel("Vídeo")
plt.ylabel("Quantidade")
plt.tight_layout()
plt.show()
    

In [ ]:

# ============================================================
# 8️⃣ Exportação do resultado
# ============================================================
OUTPUT_PATH = os.getenv("OUTPUT_PATH", "comentarios_analisados.csv")
df_pt.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")
print(f"📂 Resultados exportados: {OUTPUT_PATH}")
print(f"Linhas salvas: {len(df_pt)}")
    


---
✅ **Notebook finalizado com sucesso!**  

- Dados carregados de `comentarios_top5.csv`  
- Limpeza e normalização aplicadas  
- Idioma detectado e filtrado para **pt**  
- Modelo **BERT-PT** aplicado com **batch inference**  
- Resultados exportados em `comentarios_analisados.csv`  

📌 Na **Fase 3 (próxima aula)**, construiremos **visualizações exploratórias (EDA)** e um **dashboard interativo** com base neste resultado.
    